In [21]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

In [22]:
df = pd.read_csv(r'C:\Users\Jubin\Desktop\RP18\Datasets\intel\interpolated\injected_bias\mote=1_sensortype=temperature_faulttype=bias.csv')
df['timestamp'] = pd.to_datetime(df['timestamp']).apply(lambda x: x.timestamp()) / 10**9
X = df.drop(columns=['mote_id', 'light', 'timestamp', 'has_fault_type'])
Y = df['has_fault_type']

In [23]:
scaler = StandardScaler()
X = scaler.fit_transform(X)  # Feature scaling
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [24]:
sequence_length = 100
n_samples, input_dim = X_train.shape

X_train_sequences = []
Y_train_sequences = []

for i in range(n_samples - sequence_length + 1):
    sequence = X_train[i:i+sequence_length, :]
    label = Y_train[i+sequence_length-1]  
    X_train_sequences.append(sequence)
    Y_train_sequences.append(label)

X_train_sequences = np.array(X_train_sequences)
Y_train_sequences = np.array(Y_train_sequences)

X_train = X_train_sequences
Y_train = Y_train_sequences

sequence_length = 100
n_samples, input_dim = X_test.shape

X_test_sequences = []
Y_test_sequences = []

for i in range(n_samples - sequence_length + 1):
    sequence = X_test[i:i+sequence_length, :]  
    label = Y_test[i+sequence_length-1]  
    X_test_sequences.append(sequence)
    Y_test_sequences.append(label)

X_test_sequences = np.array(X_test_sequences)
Y_test_sequences = np.array(Y_test_sequences)

X_test = X_test_sequences
Y_test = Y_test_sequences

In [29]:
# Define the LSTM-based supervised model
input_dim = X_train.shape[2]
sequence_length = X_train.shape[1]

# Autoencoder Encoder
encoder = keras.Sequential([
    keras.layers.Input(shape=(sequence_length, input_dim)),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LSTM(32),
])

# Classifier on top of encoder
classifier = keras.Sequential([
    keras.layers.Input(shape=(32,)),  # Output dimension of the encoder
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # Binary classification output
])

# Combine the encoder and classifier
model = keras.Sequential([encoder, classifier])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model on the labeled data
model.fit(X_train, Y_train, epochs=50, batch_size=32, shuffle=True, validation_split=0.2)

In [ ]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
print(y_pred)
#y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

In [39]:
for i in range(y_pred.size):
    if y_pred[i] == 1:
        y_pred[i] = 4

In [40]:
y_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [4.],
       [0.],
       [4.]], dtype=float32)

In [41]:
confusion = confusion_matrix(Y_test, y_pred)
print("Confusion Matrix:\n", confusion)
report = classification_report(Y_test, y_pred)
print("Classification Report:\n", report)

Confusion Matrix:
 [[3021  201]
 [   0  711]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.94      0.97      3222
           4       0.78      1.00      0.88       711

    accuracy                           0.95      3933
   macro avg       0.89      0.97      0.92      3933
weighted avg       0.96      0.95      0.95      3933

